In [ ]:

symbols = ["H", "H", "H"]


def h3_ground_energy(bond_length):
    
    """
    Uses VQE to calculate the ground energy of the H3+ molecule with the given bond length.
    
    Args:
        - bond_length(float): The bond length of the H3+ molecule modelled as an
        equilateral triangle.
    Returns:
        - Union[float, np.tensor, np.array]: A float-like output containing the ground 
        state of the H3+ molecule with the given bond length.
    """
    

    dev = qml.device("default.qubit", wires=3)

    symbols, coordinates = (['H', 'H', 'H'], np.array([0.0, 0.0, 0.0, bond_length, 0.0, 0.0, bond_length / 2, bond_length * np.sqrt(3) / 2, 0.0]))

    hamiltonian, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, charge=1)


    # The Hartree-Fock State
    hf = qml.qchem.hf_state(electrons=2, orbitals=6)

    # Define the device, using lightning.qubit device
    dev = qml.device("lightning.qubit", wires=qubits)

    @qml.qnode(dev, diff_method="adjoint")
    def cost_func(params):
        qml.BasisState(hf, wires=range(qubits))
        qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])
        qml.DoubleExcitation(params[1], wires=[0, 1, 4, 5])
        return qml.expval(hamiltonian)

    def workflow(params, ntrials):
        opt = qml.GradientDescentOptimizer(stepsize=0.4)

        for n in range(ntrials):
            params, prev_energy = opt.step_and_cost(cost_func, params)
            val = cost_func(params)

        return val

    theta = workflow(np.array([0.0, 0.0]), 10)
    
    return theta

# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    outs = h3_ground_energy(ins)
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output,expected_output, atol = 1e-4), "Not the correct ground energy"